# 5章 エラー、ログ、デバッグ

## 5.1 Pythonのエラー

### 5.1.1 Pythonのエラーメッセージ

In [1]:
from scipy.stats import linregress


def fit_trendline(year_timestamps):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared

In [2]:
# エラーが出るので一旦コメントアウト
# timestamps=[2000, 2001, 2002]
# fit_trendline(timestamps)

上のコードは以下のエラーを引き起こす
```text
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[2], line 2
      1 timestamps=[2000, 2001, 2002]
----> 2 fit_trendline(timestamps)

Cell In[1], line 5
      4 def fit_trendline(year_timestamps):
----> 5     result = linregress(year_timestamps, data)
      6     slope = round(result.slope, 3)
      7     r_squared = round(result.rvalue**2, 3)

NameError: name 'data' is not defined
```

### 5.1.2 エラー処理

```python
try:
    <何らかの処理をするコード>  # ← tryブロック
except KeyError:
    <このタイプのエラーが起きてしまったときだけに実行したいコード>  # exceptブロック
```

```python
except (KeyError, ValueError):
```

graph TD;
	A[tryブロックのコード]-->B[エラーが発生しない];
	B-->D[elseブロックのコードを実行];
	A-->C[エラーが発生する];
	C-->E[exceptブロックのコードを実行];
	D-->F[finalyブロックのコードを実行];
	C-->F;

5.1.1で使用したtrendline関数を用いて、正しい引数でエラー処理をみる

In [3]:
from scipy.stats import linregress

def fit_trendline(year_timestamps, data):
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared

例外を起こさせる（**スロー**させる）ために、不正な入力で実行する

In [4]:
# women_in_parliament = [9.02, 9.01, 8.84, 8.84, 8.84]
# timestamps=["2000","2001","2002","2003","2004","2005"]
# fit_trendline(timestamps,women_in_parliament)

`women_in_parliament`と`timestamp`いずれのリストも浮動小数点数か整数で構成されていなければならないため、ValueErrorが起こる。  
このエラーは、`try`、`except`、`else`のブロックを使って、エラーを出さずにコードが実行されるように処理できる。

In [5]:
def fit_trendline(year_timestamps, data):
    try:
        result = linregress(year_timestamps, data)
    except ValueError:
        print(f"*ERROR*:Both listsmust containonly floatorintegers")
    else:
        slope = round(result.slope, 3)
        r_squared = round(result.rvalue**2, 3)
        return slope, r_squared

また、`except`ブロックでデフォルト値を返すこともできる。  
→ 関数の実行が成功した場合に返す値と同じ形式にすることで、関数のインターフェースの一貫性を保つことができる。

In [6]:
def fit_trendline(year_timestamps, data):
    try:
        result = linregress(year_timestamps, data)
    except ValueError:
        print(f"*ERROR*:Both listsmust containonly floatorintegers")
        return 0.0, 0.0
    else:
        slope = round(result.slope, 3)
        r_squared = round(result.rvalue**2, 3)
        return slope, r_squared

In [7]:
women_in_parliament = [9.02, 9.01, 8.84, 8.84, 8.84]
timestamps=["2000","2001","2002","2003","2004","2005"]
fit_trendline(timestamps, women_in_parliament)

*ERROR*:Both listsmust containonly floatorintegers


(0.0, 0.0)

### 5.1.3 エラーの生成

In [8]:
from scipy.stats import linregress


def fit_trendline(year_timestamps, data):
    if not year_timestamps or data:
        raise ValueError("Timestamps and data cannot be empty lists")
    result = linregress(year_timestamps, data)
    slope = round(result.slope, 3)
    r_squared = round(result.rvalue**2, 3)
    return slope, r_squared

In [9]:
fit_trendline([], [18.36, 18.36, 17.91])

ValueError: Timestamps and data cannot be empty lists

## 5.2　ロギング

### 5.2.1 何を記録するか

- 長時間実行中のタスクが開始または終了したことを示すメッセージ
- 本番システムでどのような問題が起こったのかを知るためのエラーメッセージ
- 特定の関数をどの関数が呼び出したか
- 特定の関数の入力と出力
- データが保存されたファイルのパス

### 5.2.2 ロギングの設定

Pythonのログレベル

| レベル | 説明 |
|----|----|
| DEBUG | デバッグ。<br>詳細な情報であり、通常は問題を診断するときにのみ役立つ |
| INFO | 情報。<br>物事が期待通りに動いていることを確認する |
| WARNING | 警告。<br>予期せぬ事が起こった。または近い将来に何らかの問題が起こる可能性があることを示す（例えば「ディスク領域が少ない」など）。<br>ソフトウェアは期待通りに動いている |
| ERROR | エラー。<br>より深刻な問題により、ソフトウェアが何らかの機能を実行できなくなっている |
| CRITICAL | 深刻なエラー。<br>プログラム自体が実行を継続できない可能性があることを示す |

Pythonの`logging`モジュールのデフォルトのロギングレベルはWARNINGである。  
 → ロギングレベルを変更せずに`INFO`レベルのメッセージを書いた場合、**そのメッセージは保存されない**

何らかのログを記録したい場合は、まずレベルをセットする。

```python
import logging


logging.basicConfig(level=logging.DEBUG)
```

もう1つのデフォルト設定でよく指定されるものに、ログの保存場所がある。  
デフォルトではコンソールにログを出力するが、ファイルに保存しておくとコードの実行が終わった後に参照できて便利。  
ログを別ファイルに保存したい場合は、次の設定を追加する。
```python
logging.basicConfig(filename='chapter_5_logs.log', level=logging.DEBUG)
```

ファイルに書き込むときのデフォルトの設定は、コードが実行されるたびにログをファイルに追加する。  
ファイルを上書きしたい場合は、`filemode='w'`を指定する。
```python
logging.basicConfig(filename='chapter_5_logs.log', filemode='w',
                    level=logging.DEBUG)
```

### 5.2.3 ログの取り方

In [10]:
import logging
from scipy.stats import linregress


logging.basicConfig(filename='chapter_5_logs.log', filemode='w',
                    level=logging.DEBUG,
                    format='%(asctime)s %(message)s')

def fit_trendline(year_timestamps, data):
    logging.info("Running fit_trendline function")
    try:
        result = linregress(year_timestamps, data)
    except TypeError as e:
        logging.error("Both lists must contain floats or integers.")
        logging.exception(e)
    else:
        slope = round(result.slope, 3)
        r_squared = round(result.rvalue**2, 3)
        logging.info(f"Completed analysis. Slope of the trendline is {slope}.")
        return slope, r_squared

In [11]:
women_in_parliament=[9.02, 9.01, 8.84, 8.84, 8.84]
timestamps= [2000, 2001, 2002, 2003, 2004]
fit_trendline(timestamps, women_in_parliament)

(np.float64(-0.053), np.float64(0.763))

## 5.3 デバッグ

### 5.3.1 デバッグの戦略

### 5.3.2 デバッグ用ツール

In [12]:
import logging


logging.basicConfig(filename='ch05_logs.log',
                    level=logging.DEBUG,
                    filemode='w',
                    format='%(asctime)s %(message)s')

def weighted_mean(num_list, weights):
    running_total = 0
    for i in range(len(num_list)):
        running_total += (num_list[i] * weights[0])
        logging.debug(f"The running total at step {i} is {running_total}")

    return (running_total / len(num_list))

## 5.4 まとめ